# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096790


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:13,  2.62s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:17,  2.86s/it]

Rendering models:  13%|█▎        | 4/30 [00:09<01:02,  2.39s/it]

Rendering models:  17%|█▋        | 5/30 [00:10<00:46,  1.88s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:20<00:01,  1.91it/s]

Rendering models:  93%|█████████▎| 28/30 [00:20<00:01,  1.81it/s]

Rendering models:  97%|█████████▋| 29/30 [00:21<00:00,  1.61it/s]

Rendering models: 100%|██████████| 30/30 [00:22<00:00,  1.67it/s]

not-logged-in-3b0b929f4475fd276561    0.000583
not-logged-in-673e227f920b24558cc1    0.000519
mwalsh868                             0.000234
jnarayanbvg                           0.020327
Galactea                              0.003680
not-logged-in-0707d430a0610af06bb6    0.022137
awright5                              0.000631
framosuw                              0.000385
not-logged-in-85c1a59add3d7b7fbef1    0.019819
not-logged-in-e8b405843217e2525550    0.011082
CScampstudent                         0.001049
redboy21                              0.107777
sn346808                              0.000289
engrg                                 0.000307
ayluminati                            0.002852
Tenorclef                             0.000471
Frosst.matt                           0.001323
KevinKosa                             0.000372
not-logged-in-a7658ef4de3eefd90ccb    0.002954
plankjames                            0.000240
not-logged-in-14d09f70263c1fe15cc3    0.000458
Mongin       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())